Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128
hl_num = 1024 #Number of neuron in hidden layers

def weight_variable(shape):
  initial = tf.truncated_normal(shape) #, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  #initial = tf.constant(0.1, shape=shape)
  #return tf.Variable(initial)
  return tf.Variable(tf.zeros([shape[0]]))


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  # biases = tf.Variable(tf.zeros([num_labels]))

  #start with hidden layer random initialization of variables and biases with a low values
  wh1 = weight_variable([image_size*image_size,hl_num])
  bh1 = bias_variable([hl_num])

  # Variables output layer
  wo = weight_variable([hl_num, num_labels])
  bo = bias_variable([num_labels])

  # Forward Propagation
  def forward_prop(input):
    h1 = tf.nn.relu(tf.matmul(input, wh1)+bh1)
    return tf.matmul(h1,wo) + bo
  
  # Training computation.
  logits =forward_prop(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # L2 Regularization 
  regularizers = tf.nn.l2_loss(wh1) + tf.nn.l2_loss(bh1) + tf.nn.l2_loss(wo) + tf.nn.l2_loss(bo)
  #regularizers = tf.nn.l2_loss(wh1) + tf.nn.l2_loss(wo) 
  
  # Add the regularization to the loss
  loss += 5e-3 * regularizers
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

In [6]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 1913.099121
Minibatch accuracy: 7.8%
Validation accuracy: 17.8%
Minibatch loss at step 500: 957.786255
Minibatch accuracy: 85.9%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 578.959290
Minibatch accuracy: 82.0%
Validation accuracy: 82.1%
Minibatch loss at step 1500: 347.772797
Minibatch accuracy: 88.3%
Validation accuracy: 81.3%
Minibatch loss at step 2000: 210.810013
Minibatch accuracy: 87.5%
Validation accuracy: 82.7%
Minibatch loss at step 2500: 127.611641
Minibatch accuracy: 85.9%
Validation accuracy: 84.5%
Minibatch loss at step 3000: 77.319153
Minibatch accuracy: 86.7%
Validation accuracy: 85.8%
Minibatch loss at step 3500: 47.110161
Minibatch accuracy: 87.5%
Validation accuracy: 86.2%
Minibatch loss at step 4000: 28.683954
Minibatch accuracy: 87.5%
Validation accuracy: 86.9%
Minibatch loss at step 4500: 17.618158
Minibatch accuracy: 89.1%
Validation accuracy: 86.6%
Minibatch loss at step 5000: 10.904655
Minibatch accuracy: 89.1%
V

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
num_steps = 3

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1992.241455
Minibatch accuracy: 7.8%
Validation accuracy: 17.2%
Minibatch loss at step 1: 1870.122925
Minibatch accuracy: 11.7%
Validation accuracy: 27.5%
Minibatch loss at step 2: 1796.171387
Minibatch accuracy: 28.9%
Validation accuracy: 33.8%
Test accuracy: 36.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?
Accuracy is increased from 36.5% to 49.0%
---

In [ ]:
num_steps = 3

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [10]:
batch_size = 128
hl_num = 1024 #Number of neuron in hidden layers

def weight_variable(shape):
  initial = tf.truncated_normal(shape) #, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  #initial = tf.constant(0.1, shape=shape)
  #return tf.Variable(initial)
  return tf.Variable(tf.zeros([shape[0]]))


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  keep_prob = tf.placeholder("float")
  
  # Variables.
  # weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  # biases = tf.Variable(tf.zeros([num_labels]))

  #start with hidden layer random initialization of variables and biases with a low values
  wh1 = weight_variable([image_size*image_size,hl_num])
  bh1 = bias_variable([hl_num])

  # Variables output layer
  wo = weight_variable([hl_num, num_labels])
  bo = bias_variable([num_labels])

  # Forward Propagation
  def forward_prop(input, isDrop=True):
    h1 = tf.nn.relu(tf.matmul(input, wh1)+bh1)
    if (isDrop):
        h1 = tf.nn.dropout(h1, keep_prob)
    return tf.matmul(h1,wo) + bo
  
  # Training computation.
  logits =forward_prop(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # L2 Regularization 
  regularizers = tf.nn.l2_loss(wh1) + tf.nn.l2_loss(bh1) + tf.nn.l2_loss(wo) + tf.nn.l2_loss(bo)
  #regularizers = tf.nn.l2_loss(wh1) + tf.nn.l2_loss(wo) 
  
  # Add the regularization to the loss
  loss += 1e-4 * regularizers
    
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Optimizer: set up a variable that's incremented once per batch and
  # controls the learning rate decay.
  global_step = tf.Variable(0)  # count the number of steps taken.
    
  learning_rate = tf.train.exponential_decay(
      0.01,                       # Base learning rate.
      global_step * batch_size,  # Current index into the dataset.
      train_labels.shape[0],     # Decay step.
      0.95,                      # Decay rate.
      staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset, False))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset, False))

num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 574.509888
Minibatch accuracy: 10.2%
Validation accuracy: 11.7%
Minibatch loss at step 500: 116.946213
Minibatch accuracy: 69.5%
Validation accuracy: 79.9%
Minibatch loss at step 1000: 113.982231
Minibatch accuracy: 71.9%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 70.917862
Minibatch accuracy: 78.9%
Validation accuracy: 82.3%
Minibatch loss at step 2000: 67.496498
Minibatch accuracy: 85.9%
Validation accuracy: 82.8%
Minibatch loss at step 2500: 55.623795
Minibatch accuracy: 80.5%
Validation accuracy: 82.9%
Minibatch loss at step 3000: 67.648514
Minibatch accuracy: 72.7%
Validation accuracy: 83.0%
Minibatch loss at step 3500: 59.123825
Minibatch accuracy: 77.3%
Validation accuracy: 83.6%
Minibatch loss at step 4000: 60.829922
Minibatch accuracy: 77.3%
Validation accuracy: 83.4%
Minibatch loss at step 4500: 55.539467
Minibatch accuracy: 78.9%
Validation accuracy: 83.5%
Minibatch loss at step 5000: 57.288212
Minibatch accuracy: 80.5%
Vali